In [45]:
import pandas as pd
import numpy as np
import plotly.express as px
from tqdm.auto import tqdm

In [46]:
df = pd.read_csv(r"/Users/yotamhadari/Desktop/SOLIO/Group_1_SOLIO2907.csv", index_col=[0,1])
group_df = pd.read_csv(r"/Users/yotamhadari/Desktop/SOLIO/Device_type_group_1.csv", index_col=[0])

In [47]:
df.head()
group_df.head()

,Device type
User ID,
1,real
2,sham
3,sham
4,sham
5,real


we want to calculate the difference between each participant's answers to the previous one.

In [48]:
def create_indiviudal_user_dfs(df):
    """Create a dictionary where keys are user_n and values and individual answer df for that user."""
    users_dfs = {}
    for user in df.index.get_level_values(0).drop_duplicates():  # Used later
        users_dfs[f"{user}"] = df.loc[df.index.get_level_values(0) == user]
    return users_dfs

In [49]:
users_dict = create_indiviudal_user_dfs(df)
user_1 = users_dict['1']
user_1.head(10)


def harvesting_mean_median(user_df, periods_to_check=30):
    """harvesting a participant's answers to a questionnaire for it's mean and median change in values"""
    user_mean_median = {}
    diff_per = 1
    while len(user_mean_median.keys()) < len(user_df.columns):
        user_diff = user_df.diff(periods=diff_per)
        user_desc = user_diff.describe()
        user_desc.replace(np.nan, 'NaN', inplace=True)
        diff_per += 1
        for column in (user_df.columns):
            
            if user_desc.loc['mean', column] != 'NaN':
                user_mean_median[column] = {'mean':user_desc.loc['mean', column], 'median': user_desc.loc['mean', column]}

        if diff_per > periods_to_check:

            for column in (user_df.columns):

                if column not in user_mean_median.keys():
                    user_mean_median[column] = {'mean': 'NEA', 'median': 'NEA'} # not enough answers
            break
    return user_mean_median

In [50]:
group_dict = {}
for group in group_df.iloc[:, 0].drop_duplicates():
    group_dict[group] = list(group_df.loc[group_df.iloc[:,0] == group].index)

group_dict

{'real': [1, 5, 7, 9, 16, 17, 21, 13, 14], 'sham': [2, 3, 4, 18, 20]}

In [51]:
df1 = pd.DataFrame(harvesting_mean_median(user_1))
df1.loc['mean']

Pain VAS            0.1
Qualitative pain    0.0
Day to day          0.0
Lifting weights     1.5
Walking             0.0
Sitting             0.5
Standing            0.0
Sleeping            0.0
Sex life            0.0
Social life        -0.5
Travel              0.0
Name: mean, dtype: float64

In [52]:
mean_dict = {}
median_dict = {}
for user, df in users_dict.items():
    mean_median = pd.DataFrame(harvesting_mean_median(df))
    mean = mean_median.loc['mean']
    median = mean_median.loc['median']
    mean_dict[int(user)] = mean
    median_dict[int(user)] = median
    

In [58]:
mean_df = pd.concat(mean_dict, axis=1)
median_df = pd.concat(median_dict, axis=1)

real_mean = mean_df.loc[:, group_dict['real']]
sham_mean = mean_df.loc[:, group_dict['sham']]

real_median = median_df.loc[:, group_dict['real']]
sham_median = median_df.loc[:, group_dict['sham']]

In [61]:
real_mean['mean_change'] = real_mean

,1,5,7,9,16,17,21,13,14
Pain VAS,0.1,-1.5,-0.3,-0.3,-0.444444,-1.0,1.3,-0.1,-0.5
Qualitative pain,0.0,-0.5,0.0,-0.5,0.000000,0.0,0.0,0.0,-0.5
Day to day,0.0,0.5,0.0,-0.5,0.000000,0.5,0.0,-0.5,0.0
Lifting weights,1.5,-0.5,0.5,0.0,0.000000,0.0,0.0,0.0,0.5
Walking,0.0,-1.5,0.0,-0.5,0.000000,0.0,0.0,0.0,0.0
Sitting,0.5,0.0,0.0,0.5,-1.000000,0.5,-0.5,0.5,0.0
Standing,0.0,-1.0,0.0,0.5,0.000000,-0.5,-0.5,0.0,0.0
Sleeping,0.0,0.0,0.0,0.5,0.000000,0.0,1.0,0.0,0.0
Sex life,0.0,-1.5,0.0,-0.5,0.000000,-0.5,0.0,0.0,0.0
Social life,-0.5,-0.5,-0.5,0.0,1.000000,0.0,0.0,0.0,0.0
